In [1]:
import os
import sys
import pickle

current_dir =  os.getcwd()
#parent_dir = os.path.dirname(current_dir)
#sys.path.insert(0, parent_dir)

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from cities.utils.cleaning_utils import standardize_and_scale

import matplotlib.pyplot as plt



from cities.utils.data_grabber import DataGrabber
data = DataGrabber()
data.get_features_wide(["gdp", "population", "transport",
            "spending_transportation", "spending_commerce", "spending_HHS"])
gdp = data.wide['gdp']
# population = data.wide['population']
# trnasport = data.wide['transport']
# spending_transportation = data.wide['spending_transportation']
# spending_commerce = data.wide['spending_commerce']

def tableInfo(tableName):

    print(tableName.head())
    print(tableName.dtypes)
    print(f'Number of rows: {tableName.shape[0]}')
    print(f'Unique FIPS numbers {tableName['GeoFIPS'].nunique()}')

In [ ]:
# dtype_mapping = {'STATE': str, 'COUNTY': str}
# urbanization = pd.read_csv(f"{grandparent_dir}/data/raw/2020_UA_COUNTY.csv", dtype=dtype_mapping)

In [2]:
industry_ts = pd.read_csv(f"{grandparent_dir}/data/raw/industry_time_series_people.csv")

industry_ts["GEO_ID"] = industry_ts["GEO_ID"].str.split("US").str[1]
industry_ts["GEO_ID"] = industry_ts["GEO_ID"].astype("int64")
industry_ts = industry_ts.rename(columns={"GEO_ID": "GeoFIPS"})

common_fips = np.intersect1d(gdp["GeoFIPS"].unique(), industry_ts["GeoFIPS"].unique())


industry_ts = industry_ts[industry_ts["GeoFIPS"].isin(common_fips)]


years = industry_ts['Year'].unique()

for year in years:
    year_df = industry_ts[industry_ts['Year'] == year]
    missing_fips = set(common_fips) - set(year_df['GeoFIPS'])
    
    if missing_fips:
        missing_data = {'Year': [year] * len(missing_fips), 'GeoFIPS': list(missing_fips)}
        
        # Fill all columns from the fourth column (index 3) onward with 0
        for col in industry_ts.columns[2:]:
            missing_data[col] = 0
        
        missing_df = pd.DataFrame(missing_data)
        industry_ts = pd.concat([industry_ts, missing_df], ignore_index=True)

industry_ts = industry_ts.merge(gdp[["GeoFIPS", "GeoName"]], on="GeoFIPS", how="left")


industry_ts = industry_ts[
    [
        "GeoFIPS",
        "GeoName",
        "Year",
        'agriculture_total',
        'mining_total',
        'construction_total',
        'manufacturing_total',
        'wholesale_trade_total',
        'retail_trade_total',
        'transportation_warehousing_total',
        'utilities_total',
        'information_total',
        'finance_insurance_total',
        'real_estate_total',
        'professional_services_total',
        'management_enterprises_total',
        'admin_support_services_total',
        'educational_services_total',
        'healthcare_social_services_total',
        'arts_recreation_total',
        'accommodation_food_services_total',
        'other_services_total',
        'public_administration_total',
    ]
]

industry_ts = industry_ts.sort_values(by=["GeoFIPS", "GeoName", "Year"])

In [5]:
missing_values = industry_ts.isnull().sum()

missing_values

GeoFIPS                              0
GeoName                              0
Year                                 0
agriculture_total                    0
mining_total                         0
construction_total                   0
manufacturing_total                  0
wholesale_trade_total                0
retail_trade_total                   0
transportation_warehousing_total     0
utilities_total                      0
information_total                    0
finance_insurance_total              0
real_estate_total                    0
professional_services_total          0
management_enterprises_total         0
admin_support_services_total         0
educational_services_total           0
healthcare_social_services_total     0
arts_recreation_total                0
accommodation_food_services_total    0
other_services_total                 0
public_administration_total          0
dtype: int64

In [115]:
columns_to_save = industry_ts.columns[industry_ts.columns.get_loc('Year') + 1:]


for column in columns_to_save:

    selected_columns = ['GeoFIPS', 'GeoName', 'Year', column]
    subsetindustry_ts = industry_ts[selected_columns]

    subsetindustry_ts.rename(columns={column: 'Value'}, inplace=True)
    

    subsetindustry_ts_long = subsetindustry_ts.copy()

    file_name_long = f"industry_{column}_long.csv"
    subsetindustry_ts_long.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_long}", index=False)



    subsetindustry_ts_std_long = standardize_and_scale(subsetindustry_ts)

    file_name_std = f"industry_{column}_std_long.csv"
    subsetindustry_ts_std_long.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_std}", index=False)


    subsetindustry_ts_wide = subsetindustry_ts.pivot_table(
    index=["GeoFIPS", "GeoName"], columns="Year", values="Value"
    )
    subsetindustry_ts_wide.reset_index(inplace=True)
    subsetindustry_ts_wide.columns.name = None

    
    file_name_wide = f"industry_{column}_wide.csv"
    subsetindustry_ts_wide.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_wide}", index=False)
    
    

    subsetindustry_ts_std_wide = standardize_and_scale(subsetindustry_ts_wide)

    assert (subsetindustry_ts_std_wide[column] >= -1).all() and subsetindustry_ts_std_wide[column] <= 1).all(), 'STD ERROR'



    file_name_std_wide = f"industry_{column}_std_wide.csv"
    subsetindustry_ts_std_wide.to_csv(f"{grandparent_dir}/data/processed/dump/{file_name_std_wide}", index=False)

    # first standardize then save

/tmp/ipykernel_13900/2843878824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsetindustry_ts.rename(columns={column: 'Value'}, inplace=True)


KeyError: 'agriculture_total'

In [ ]:
assert (data.std_wide[feature][column] >= -1).all() and (
                    data.std_wide[feature][column] <= 1
                ).all(), std_error

In [108]:
columns_to_check = industry_ts.columns[3:]

# Check for values outside the range [0, 1,000,000] in these columns
values_outside_range = industry_ts[columns_to_check].apply(lambda x: (x < 0) | (x > 1000000))

# Display rows where values are outside the range
rows_with_values_outside_range = industry_ts[values_outside_range.any(axis=1)]

In [109]:
rows_with_values_outside_range

,GeoFIPS,GeoName,Year,agriculture_total,mining_total,construction_total,manufacturing_total,wholesale_trade_total,retail_trade_total,transportation_warehousing_total,...,real_estate_total,professional_services_total,management_enterprises_total,admin_support_services_total,educational_services_total,healthcare_social_services_total,arts_recreation_total,accommodation_food_services_total,other_services_total,public_administration_total


In [4]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

loaded_data


with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

# Save the loaded data to a CSV file
#loaded_data.to_csv('your_output_file.csv', index=False) 

{'transport': array([ 2282, 15901, 22051, 22071, 22075, 22087, 22089, 22095, 22103,
        51901, 51903, 51907, 51911, 51913, 51918, 51919, 51921, 51923,
        51929, 51931, 51933, 51939, 51941, 51942, 51944, 51945, 51947,
        51949, 51951, 51953, 51955, 51958])}

In [6]:
with open(f"{grandparent_dir}/data/raw/exclusions.pkl", 'rb') as file:
    loaded_data = pickle.load(file)


loaded_data

{'transport': array([ 2282, 15901, 22051, 22071, 22075, 22087, 22089, 22095, 22103,
        51901, 51903, 51907, 51911, 51913, 51918, 51919, 51921, 51923,
        51929, 51931, 51933, 51939, 51941, 51942, 51944, 51945, 51947,
        51949, 51951, 51953, 51955, 51958])}

In [12]:
df = pd.DataFrame.from_dict(loaded_data)  # 'orient' specifies that the keys should be used as the index

df


melted_df = pd.melt(df, var_name='dataset', value_name='exclusions')
melted_df['dataset'] = 'transport'
melted_df

,dataset,exclusions
0,transport,2282
1,transport,15901
2,transport,22051
3,transport,22071
4,transport,22075
5,transport,22087
6,transport,22089
7,transport,22095
8,transport,22103
9,transport,51901


In [44]:
industry_ts 

,GeoFIPS,GeoName,Year,agriculture_total,mining_total,construction_total,manufacturing_total,wholesale_trade_total,retail_trade_total,transportation_warehousing_total,...,real_estate_total,professional_services_total,management_enterprises_total,admin_support_services_total,educational_services_total,healthcare_social_services_total,arts_recreation_total,accommodation_food_services_total,other_services_total,public_administration_total
1067,1001,"Autauga, AL",2010,0.006738,0.002711,0.077400,0.141284,0.031141,0.126659,0.032743,...,0.017419,0.045027,0.000246,0.024691,0.068567,0.105748,0.017419,0.069348,0.045520,0.116676
4180,1001,"Autauga, AL",2011,0.010150,0.002320,0.069804,0.139650,0.036290,0.124984,0.042255,...,0.016902,0.043042,0.000000,0.027466,0.075314,0.107834,0.016778,0.064833,0.044285,0.112722
9238,1001,"Autauga, AL",2012,0.010860,0.003482,0.056456,0.135876,0.029720,0.134964,0.044933,...,0.016456,0.043026,0.000000,0.031171,0.072746,0.115233,0.019150,0.058860,0.044684,0.114031
10327,1001,"Autauga, AL",2013,0.010193,0.003843,0.058027,0.128838,0.030163,0.135272,0.048502,...,0.015666,0.046831,0.000000,0.031374,0.063333,0.113757,0.019092,0.056523,0.059114,0.113924
13388,1001,"Autauga, AL",2014,0.011349,0.004391,0.056124,0.132295,0.031976,0.127863,0.040881,...,0.015118,0.049704,0.000000,0.031852,0.063041,0.115934,0.017686,0.058402,0.061550,0.112993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23555,56045,"Weston, WY",2017,0.053713,0.192545,0.052245,0.061638,0.012328,0.112122,0.061344,...,0.002348,0.011154,0.000000,0.017317,0.125037,0.093044,0.005283,0.056648,0.025829,0.060757
25620,56045,"Weston, WY",2018,0.048934,0.203890,0.065558,0.050816,0.012861,0.094417,0.073714,...,0.001882,0.016311,0.000000,0.018193,0.115747,0.091280,0.008469,0.053011,0.021330,0.057403
28703,56045,"Weston, WY",2019,0.062093,0.188140,0.049674,0.054331,0.012729,0.093760,0.064266,...,0.004346,0.025147,0.000000,0.022043,0.113940,0.081341,0.008072,0.054020,0.049053,0.064887
32913,56045,"Weston, WY",2020,0.046048,0.163918,0.072509,0.069759,0.010309,0.100344,0.048454,...,0.020619,0.023368,0.000000,0.021649,0.079725,0.101375,0.018557,0.029897,0.053952,0.049485
